# Phase 2 - Joining the data
Trevor's [Phase 1](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/1038316885899254/command/1038316885899255) and [Phase 2](https://adb-731998097721284.4.azuredatabricks.net/?o=731998097721284#notebook/3816960191357033/command/3816960191357034) explorations were used as a base
<br> Architecture diagram courtesy of Esther

## Join Architecture Diagram
Below, we show the planned sequence of joins to join our 4 datasets (OpenFights, Stations, Airports, Weather) into the full dataset we plan to use for modeling.

This first diagram shows how we dealed with the missing airport and generated a mapping table containing closest single weather station for each airport.

![Joining_Process_1](files/shared_uploads/yuqiaochen@berkeley.edu/Joining_Process_1.png)

This second diagram shows all the joins we have performed to get our final fully joined dataset.

![Joining_Process_2](files/shared_uploads/yuqiaochen@berkeley.edu/Joining_Process_2.png)

## Imports

In [0]:
from pyspark.sql import types, functions as F
import pandas as pd
from pyspark.sql.functions import col, max
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

## Configure Databricks blob access

In [0]:
blob_container = "main-storage" # The name of your container created in https://portal.azure.com
storage_account = "team05w261" # The name of your Storage account created in https://portal.azure.com
secret_scope = "team05" # The name of the scope created in your local computer using the Databricks CLI
secret_key = "team05-key" # The name of the secret key created in your local computer using the Databricks CLI 
blob_url = f"wasbs://{blob_container}@{storage_account}.blob.core.windows.net"
mount_path = "/mnt/mids-w261"

# Configure blob storage account access key globally
spark.conf.set(
  f"fs.azure.account.key.{storage_account}.blob.core.windows.net",
  dbutils.secrets.get(scope = secret_scope, key = secret_key)
)

In [0]:
display(dbutils.fs.ls(f"{mount_path}/datasets_final_project"))

path,name,size
dbfs:/mnt/mids-w261/datasets_final_project/airlines/,airlines/,0
dbfs:/mnt/mids-w261/datasets_final_project/airlines_data/,airlines_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data/,parquet_airlines_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data_3m/,parquet_airlines_data_3m/,0
dbfs:/mnt/mids-w261/datasets_final_project/parquet_airlines_data_6m/,parquet_airlines_data_6m/,0
dbfs:/mnt/mids-w261/datasets_final_project/stations_data/,stations_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data/,weather_data/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data_6_hr/,weather_data_6_hr/,0
dbfs:/mnt/mids-w261/datasets_final_project/weather_data_single/,weather_data_single/,0


## Read in datasets

In [0]:
###########################################
##### For FULL dataset, run this cell #####
###########################################

df_airlines = spark.read.parquet("/mnt/mids-w261/datasets_final_project/parquet_airlines_data/*")
df_stations = spark.read.parquet('dbfs:/mnt/mids-w261/datasets_final_project/stations_data/*')
df_weather = spark.read.parquet('dbfs:/mnt/mids-w261/datasets_final_project/weather_data/*')
open_flights = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat", 
                           names = ['id','name', 'city', 'country', 'iata', 'icao', 'lat', 'lng', 'altitude', 
                                    'timezone', 'dst', 'tz_db_time_zone', 'type', 'source'])
open_flights = spark.createDataFrame(open_flights)

In [0]:
##############################################
##### For LIMITED dataset, run this cell #####
##############################################

df_airlines = spark.read.parquet("/mnt/mids-w261/datasets_final_project/parquet_airlines_data_3m/")
df_stations = spark.read.parquet('dbfs:/mnt/mids-w261/datasets_final_project/stations_data/*')
df_weather = spark.read.parquet('dbfs:/mnt/mids-w261/datasets_final_project/weather_data/*').filter(F.col('DATE') < "2015-04-01T00:00:00.000")
open_flights = pd.read_csv("https://raw.githubusercontent.com/jpatokal/openflights/master/data/airports.dat", 
                           names = ['id','name', 'city', 'country', 'iata', 'icao', 'lat', 'lng', 'altitude', 
                                    'timezone', 'dst', 'tz_db_time_zone', 'type', 'source'])
open_flights = spark.createDataFrame(open_flights)

In [0]:
# Full dataset row count

# df_airlines.count() # 63493682 rows
# df_stations.count() # 5004169 rows
# df_weather.count() # 630904436 rows
# open_flights.count() # 7698 rows

Out[14]: 7698

## Subset columns to those we are keeping and remove duplicates

In [0]:
# list the columns we're keeping
weather_col = ['STATION', 'SOURCE', 'DATE', 'LATITUDE', 'LONGITUDE', 'ELEVATION', 
               'NAME', 'REPORT_TYPE', 'CALL_SIGN', 'QUALITY_CONTROL', 'WND', 
               'CIG', 'VIS', 'TMP', 'DEW', 'SLP', 'GA1', 'GF1', 'MA1', 'REM', 
               'AA1', 'AA2', 'AJ1', 'AL1', 'AN1', 'AO1', 'AU1', 'AT1']
airline_col = ["YEAR", "QUARTER", "MONTH", "DAY_OF_MONTH", "DAY_OF_WEEK", 
               "FL_DATE", "DEP_TIME", "DEP_TIME_BLK", "CRS_DEP_TIME", "CRS_ARR_TIME",
               "CRS_ELAPSED_TIME", "ARR_TIME", "ARR_TIME_BLK", "ACTUAL_ELAPSED_TIME",
               "ORIGIN", "ORIGIN_CITY_NAME", "ORIGIN_STATE_ABR", "ORIGIN_STATE_FIPS",
               "ORIGIN_STATE_NM", "ORIGIN_WAC",
               "DEST", "DEST_CITY_NAME", "DEST_STATE_ABR", "DEST_STATE_FIPS", 
               "DEST_STATE_NM","DEST_WAC",
               "OP_UNIQUE_CARRIER", "FLIGHTS", "DISTANCE", "DISTANCE_GROUP", "DIVERTED",
               "CANCELLED", "CANCELLATION_CODE", "CARRIER_DELAY", "DEP_DELAY", "DEP_DELAY_NEW",
               "DEP_DELAY_GROUP", "DEP_DEL15", "ARR_DELAY", "ARR_DELAY_NEW", "ARR_DELAY_GROUP",
               "ARR_DEL15", "WEATHER_DELAY", "NAS_DELAY", "SECURITY_DELAY", "LATE_AIRCRAFT_DELAY", "TAIL_NUM"]
station_col = ['station_id', 'wban', 'lat', 'lon', 'neighbor_id', 'neighbor_name', 'neighbor_state',
               'neighbor_call', 'neighbor_lat', 'neighbor_lon', 'distance_to_neighbor']
openfli_col = ['name', 'city', 'country', 'iata', 'lat', 'lng', 'icao', 'timezone', 'tz_db_time_zone']

In [0]:
# perform subsetting and drop duplicates
df_airlines = df_airlines.select(airline_col).dropDuplicates()
df_weather = df_weather.select(weather_col) # We did EDA and know there's no duplicate
df_stations = df_stations.select(station_col).dropDuplicates()
open_flights = open_flights.select(openfli_col).dropDuplicates()

In [0]:
# Filter out cancelled and diverted flights
df_airlines = df_airlines.filter((df_airlines.CANCELLED == 0) & (df_airlines.DIVERTED == 0))

## Prepping for a airport_weather_mapping

In [0]:
# select subset of columns
open_flights = open_flights.select('iata', 'icao', 'lat', 'lng', 'tz_db_time_zone')
print(open_flights.count())

# add 4 more airports
open_flights2 = spark.createDataFrame(pd.DataFrame({
    'iata': ['XWA', 'EAR', 'TKI', 'IFP'], 
    'icao': ['KXWA', 'KEAR', 'KTKI', 'KIFP'], 
    'lat': [48.2578135, 40.7274925, 33.1775399, 35.16558], 
    'lng': [-103.7418471, -99.0122646, -96.5926444, -114.557093], 
    'tz_db_time_zone': ['America/Chicago', 'America/Chicago', 'America/Chicago', 'America/Phoenix']
}))

# now union them
open_flights = open_flights.union(open_flights2)

# only need to keep the flights that are in df_airlines
origin_airports = df_airlines.select("origin").distinct().toPandas()
dest_airports = df_airlines.select("dest").distinct().toPandas()
all_airports = set(list(origin_airports.origin) + list(dest_airports.dest))
# do the filter
open_flights = open_flights.filter(F.col('iata').isin(all_airports))

open_flights.cache()

7698
Out[8]: DataFrame[iata: string, icao: string, lat: double, lng: double, tz_db_time_zone: string]

In [0]:
open_flights.createOrReplaceTempView("open_flights")
# 45,638 distinct weather stations
df_weather.select('latitude', 'longitude', 'station').distinct().createOrReplaceTempView('df_weather')

open_flights_closest_weather = spark.sql('''

with 

--step1: get every single combination between the two
-- and compute distance between. just use euclidean for now
tbl1 as (
    select
        a.iata
        ,a.icao
        ,a.lat lat_of
        ,a.lng lng_of
        ,b.latitude lat_w
        ,b.longitude lng_w
        ,b.station
        ,a.tz_db_time_zone
        ,power(power(a.lat - b.latitude,2) + power(a.lng - b.longitude,2),.5) as distance_euclidean
    from open_flights a 
    cross join df_weather b
)

--step2: the memory intensive sort to sort the distances for each airport
,tbl2 as(
    select
        tbl1.*
        ,dense_rank() over (partition by tbl1.iata order by tbl1.distance_euclidean) as n
    from tbl1
)

--step 3: filter to only include the 1 closest weather station. We could alternatively filter to include the top 3 closest weather stations and use that to take some sort of average if we want. 

select *
from tbl2
where n = 1
''')

# see a view of the whole thing:
display(open_flights_closest_weather)

# select a subset of columns to use as our "master mapping"
airport_weather_mapping = open_flights_closest_weather.select(['iata', 'icao', 'station', 'tz_db_time_zone'])
airport_weather_mapping.cache()

iata,icao,lat_of,lng_of,lat_w,lng_w,station,tz_db_time_zone,distance_euclidean,n
ATY,KATY,44.91400146,-97.15470123,44.9047,-97.1494,72654614946,America/Chicago,0.010706082273388616,1
BGM,KBGM,42.20869827,-75.97979736,42.2068,-75.98,72515004725,America/New_York,0.0019090552539127372,1
BUR,KBUR,34.20069885253906,-118.35900115966795,34.20056,-118.3575,72288023152,America/Los_Angeles,0.0015075677020590299,1
DLG,PADL,59.04470062,-158.5050049,59.05,-158.5167,70321025513,America/Anchorage,0.012839734903583469,1
DRT,KDRT,29.3742008209,-100.927001953,29.3784,-100.927,72261022010,America/Chicago,0.004199179554159913,1
EUG,KEUG,44.12459945678711,-123.21199798583984,44.1278,-123.2206,72693024221,America/Los_Angeles,0.00917813295116121,1
GEG,KGEG,47.61989974975586,-117.53399658203124,47.6216,-117.528,72785024157,America/Los_Angeles,0.006232964539444569,1
GRB,KGRB,44.48509979248047,-88.12960052490234,44.4794,-88.1366,72645014898,America/Chicago,0.009026643117081632,1
GRR,KGRR,42.88079834,-85.52279663,42.8825,-85.52389,72635094860,America/New_York,0.002022647945760716,1
GTF,KGTF,47.48199844,-111.3710022,47.4733,-111.3822,72775024143,America/Denver,0.014179336489177843,1


Out[9]: DataFrame[iata: string, icao: string, station: string, tz_db_time_zone: string]

## Prepping for join part 1

In [0]:
# Adding an ID to the flights data
df_airlines = df_airlines.withColumn("id", F.monotonically_increasing_id())

# rename airports columns with suffix
old_col_nms = df_airlines.columns
new_col_nms = [var + '_AIRLNS' for var in old_col_nms]
for i in range(len(old_col_nms)):
  df_airlines = df_airlines.withColumnRenamed(old_col_nms[i], new_col_nms[i])
df_airlines.columns

Out[10]: ['YEAR_AIRLNS',
 'QUARTER_AIRLNS',
 'MONTH_AIRLNS',
 'DAY_OF_MONTH_AIRLNS',
 'DAY_OF_WEEK_AIRLNS',
 'FL_DATE_AIRLNS',
 'DEP_TIME_AIRLNS',
 'DEP_TIME_BLK_AIRLNS',
 'CRS_DEP_TIME_AIRLNS',
 'CRS_ARR_TIME_AIRLNS',
 'CRS_ELAPSED_TIME_AIRLNS',
 'ARR_TIME_AIRLNS',
 'ARR_TIME_BLK_AIRLNS',
 'ACTUAL_ELAPSED_TIME_AIRLNS',
 'ORIGIN_AIRLNS',
 'ORIGIN_CITY_NAME_AIRLNS',
 'ORIGIN_STATE_ABR_AIRLNS',
 'ORIGIN_STATE_FIPS_AIRLNS',
 'ORIGIN_STATE_NM_AIRLNS',
 'ORIGIN_WAC_AIRLNS',
 'DEST_AIRLNS',
 'DEST_CITY_NAME_AIRLNS',
 'DEST_STATE_ABR_AIRLNS',
 'DEST_STATE_FIPS_AIRLNS',
 'DEST_STATE_NM_AIRLNS',
 'DEST_WAC_AIRLNS',
 'OP_UNIQUE_CARRIER_AIRLNS',
 'FLIGHTS_AIRLNS',
 'DISTANCE_AIRLNS',
 'DISTANCE_GROUP_AIRLNS',
 'DIVERTED_AIRLNS',
 'CANCELLED_AIRLNS',
 'CANCELLATION_CODE_AIRLNS',
 'CARRIER_DELAY_AIRLNS',
 'DEP_DELAY_AIRLNS',
 'DEP_DELAY_NEW_AIRLNS',
 'DEP_DELAY_GROUP_AIRLNS',
 'DEP_DEL15_AIRLNS',
 'ARR_DELAY_AIRLNS',
 'ARR_DELAY_NEW_AIRLNS',
 'ARR_DELAY_GROUP_AIRLNS',
 'ARR_DEL15_AIRLNS',
 'WEATHER_DELAY_AIRLNS',
 'NAS_DELAY_AIRLNS',
 'SECURITY_DELAY_AIRLNS',
 'LATE_AIRCRAFT_DELAY_AIRLNS',
 'TAIL_NUM_AIRLNS',
 'id_AIRLNS']

In [0]:
# rename stations columns with suffix
old_col_nms = df_stations.columns
new_col_nms = [var + '_STNS' for var in old_col_nms]
for i in range(len(old_col_nms)):
  df_stations = df_stations.withColumnRenamed(old_col_nms[i], new_col_nms[i])
df_stations.columns


Out[11]: ['station_id_STNS',
 'wban_STNS',
 'lat_STNS',
 'lon_STNS',
 'neighbor_id_STNS',
 'neighbor_name_STNS',
 'neighbor_state_STNS',
 'neighbor_call_STNS',
 'neighbor_lat_STNS',
 'neighbor_lon_STNS',
 'distance_to_neighbor_STNS']

In [0]:
# rename weather columns with suffix
old_col_nms = df_weather.columns
new_col_nms = [var + '_WTHR' for var in old_col_nms]
for i in range(len(old_col_nms)):
  df_weather = df_weather.withColumnRenamed(old_col_nms[i], new_col_nms[i])
df_weather.columns



Out[12]: ['STATION_WTHR',
 'SOURCE_WTHR',
 'DATE_WTHR',
 'LATITUDE_WTHR',
 'LONGITUDE_WTHR',
 'ELEVATION_WTHR',
 'NAME_WTHR',
 'REPORT_TYPE_WTHR',
 'CALL_SIGN_WTHR',
 'QUALITY_CONTROL_WTHR',
 'WND_WTHR',
 'CIG_WTHR',
 'VIS_WTHR',
 'TMP_WTHR',
 'DEW_WTHR',
 'SLP_WTHR',
 'GA1_WTHR',
 'GF1_WTHR',
 'MA1_WTHR',
 'REM_WTHR',
 'AA1_WTHR',
 'AA2_WTHR',
 'AJ1_WTHR',
 'AL1_WTHR',
 'AN1_WTHR',
 'AO1_WTHR',
 'AU1_WTHR',
 'AT1_WTHR']

In [0]:
# # rename openflights columns with suffix

old_col_nms = open_flights.columns
new_col_nms = [var + '_OPNFLGHT' for var in old_col_nms]
for i in range(len(old_col_nms)):
  open_flights = open_flights.withColumnRenamed(old_col_nms[i], new_col_nms[i])
open_flights.columns

Out[13]: ['iata_OPNFLGHT',
 'icao_OPNFLGHT',
 'lat_OPNFLGHT',
 'lng_OPNFLGHT',
 'tz_db_time_zone_OPNFLGHT']

## Join part 1
  
- Join the mapping table containing closest single weather station for each airport with Weather table on StationID to get enriched Weather table with airport codes.

In [0]:
# Enrich the weather data with the airport mapping, essentially attach IATA code to each weather measurement
df_weather_origin = df_weather \
    .join(airport_weather_mapping, on=[df_weather.STATION_WTHR == airport_weather_mapping.station], how='inner')\
    .withColumn('DATE_WTHR_rounded', F.date_trunc("hour", F.col('DATE_WTHR')))

## Prepping for join part 2

In [0]:
# Copy the same data to construct the df_weather_dest dataframe and rename columns
df_weather_dest = df_weather_origin.toDF(*df_weather_origin.columns)
old_col_nms = df_weather_dest.columns
new_col_nms = [var + '_dest' for var in old_col_nms]
for i in range(len(old_col_nms)):
  df_weather_dest = df_weather_dest.withColumnRenamed(old_col_nms[i], new_col_nms[i])
  
df_weather_dest.columns

Out[15]: ['STATION_WTHR_dest',
 'SOURCE_WTHR_dest',
 'DATE_WTHR_dest',
 'LATITUDE_WTHR_dest',
 'LONGITUDE_WTHR_dest',
 'ELEVATION_WTHR_dest',
 'NAME_WTHR_dest',
 'REPORT_TYPE_WTHR_dest',
 'CALL_SIGN_WTHR_dest',
 'QUALITY_CONTROL_WTHR_dest',
 'WND_WTHR_dest',
 'CIG_WTHR_dest',
 'VIS_WTHR_dest',
 'TMP_WTHR_dest',
 'DEW_WTHR_dest',
 'SLP_WTHR_dest',
 'GA1_WTHR_dest',
 'GF1_WTHR_dest',
 'MA1_WTHR_dest',
 'REM_WTHR_dest',
 'AA1_WTHR_dest',
 'AA2_WTHR_dest',
 'AJ1_WTHR_dest',
 'AL1_WTHR_dest',
 'AN1_WTHR_dest',
 'AO1_WTHR_dest',
 'AU1_WTHR_dest',
 'AT1_WTHR_dest',
 'iata_dest',
 'icao_dest',
 'station_dest',
 'tz_db_time_zone_dest',
 'DATE_WTHR_rounded_dest']

In [0]:
# Go back and rename the origin columns
# rename some weather columns
old_col_nms = df_weather_origin.columns
new_col_nms = [var + '_origin' for var in old_col_nms]
for i in range(len(old_col_nms)):
  df_weather_origin = df_weather_origin.withColumnRenamed(old_col_nms[i], new_col_nms[i])
  
df_weather_origin.columns

Out[16]: ['STATION_WTHR_origin',
 'SOURCE_WTHR_origin',
 'DATE_WTHR_origin',
 'LATITUDE_WTHR_origin',
 'LONGITUDE_WTHR_origin',
 'ELEVATION_WTHR_origin',
 'NAME_WTHR_origin',
 'REPORT_TYPE_WTHR_origin',
 'CALL_SIGN_WTHR_origin',
 'QUALITY_CONTROL_WTHR_origin',
 'WND_WTHR_origin',
 'CIG_WTHR_origin',
 'VIS_WTHR_origin',
 'TMP_WTHR_origin',
 'DEW_WTHR_origin',
 'SLP_WTHR_origin',
 'GA1_WTHR_origin',
 'GF1_WTHR_origin',
 'MA1_WTHR_origin',
 'REM_WTHR_origin',
 'AA1_WTHR_origin',
 'AA2_WTHR_origin',
 'AJ1_WTHR_origin',
 'AL1_WTHR_origin',
 'AN1_WTHR_origin',
 'AO1_WTHR_origin',
 'AU1_WTHR_origin',
 'AT1_WTHR_origin',
 'iata_origin',
 'icao_origin',
 'station_origin',
 'tz_db_time_zone_origin',
 'DATE_WTHR_rounded_origin']

In [0]:
# create a timestamp feature
df_airlines = df_airlines.withColumn('datetime_dep', 
    F.unix_timestamp(F.concat(
        F.col('FL_DATE_AIRLNS'), 
        F.lit(' '), 
        F.lpad(F.col('CRS_DEP_TIME_AIRLNS'), 4, '0')
        ), 'yyyy-MM-dd HHmm').cast(types.TimestampType())
    )

# bring time-zones 
df_airlines_pre_join = df_airlines\
  .join(open_flights.select(['iata_OPNFLGHT', 'tz_db_time_zone_OPNFLGHT']).distinct(), on=[df_airlines.ORIGIN_AIRLNS == open_flights.iata_OPNFLGHT], how='left')\
  .withColumn("utc_dep", F.to_utc_timestamp(F.col("datetime_dep"), F.col("tz_db_time_zone_OPNFLGHT")))\
  .withColumn("utc_dep_3hrs_prior", F.col('utc_dep') - F.expr('INTERVAL 3 HOURS')) \
  .withColumn("utc_dep_3hrs_prior_rounded", F.date_trunc("hour", F.col('utc_dep_3hrs_prior'))) \
  .withColumn("utc_arrive", col("utc_dep") + (col("CRS_ELAPSED_TIME_AIRLNS") * F.expr("Interval 1 Minutes")))


In [0]:
df_airlines_pre_join.drop('iata_OPNFLGHT', 'tz_db_time_zone_OPNFLGHT')

Out[18]: DataFrame[YEAR_AIRLNS: int, QUARTER_AIRLNS: int, MONTH_AIRLNS: int, DAY_OF_MONTH_AIRLNS: int, DAY_OF_WEEK_AIRLNS: int, FL_DATE_AIRLNS: string, DEP_TIME_AIRLNS: int, DEP_TIME_BLK_AIRLNS: string, CRS_DEP_TIME_AIRLNS: int, CRS_ARR_TIME_AIRLNS: int, CRS_ELAPSED_TIME_AIRLNS: double, ARR_TIME_AIRLNS: int, ARR_TIME_BLK_AIRLNS: string, ACTUAL_ELAPSED_TIME_AIRLNS: double, ORIGIN_AIRLNS: string, ORIGIN_CITY_NAME_AIRLNS: string, ORIGIN_STATE_ABR_AIRLNS: string, ORIGIN_STATE_FIPS_AIRLNS: int, ORIGIN_STATE_NM_AIRLNS: string, ORIGIN_WAC_AIRLNS: int, DEST_AIRLNS: string, DEST_CITY_NAME_AIRLNS: string, DEST_STATE_ABR_AIRLNS: string, DEST_STATE_FIPS_AIRLNS: int, DEST_STATE_NM_AIRLNS: string, DEST_WAC_AIRLNS: int, OP_UNIQUE_CARRIER_AIRLNS: string, FLIGHTS_AIRLNS: double, DISTANCE_AIRLNS: double, DISTANCE_GROUP_AIRLNS: int, DIVERTED_AIRLNS: double, CANCELLED_AIRLNS: double, CANCELLATION_CODE_AIRLNS: string, CARRIER_DELAY_AIRLNS: double, DEP_DELAY_AIRLNS: double, DEP_DELAY_NEW_AIRLNS: double, DEP_DELAY_GROUP_AIRLNS: int, DEP_DEL15_AIRLNS: double, ARR_DELAY_AIRLNS: double, ARR_DELAY_NEW_AIRLNS: double, ARR_DELAY_GROUP_AIRLNS: int, ARR_DEL15_AIRLNS: double, WEATHER_DELAY_AIRLNS: double, NAS_DELAY_AIRLNS: double, SECURITY_DELAY_AIRLNS: double, LATE_AIRCRAFT_DELAY_AIRLNS: double, TAIL_NUM_AIRLNS: string, id_AIRLNS: bigint, datetime_dep: timestamp, utc_dep: timestamp, utc_dep_3hrs_prior: timestamp, utc_dep_3hrs_prior_rounded: timestamp, utc_arrive: timestamp]

In [0]:
# df_airlines_pre_join.createOrReplaceTempView("df_airlines")
# df_weather_origin.createOrReplaceTempView("df_weather_origin")
# df_weather_dest.createOrReplaceTempView("df_weather_dest")


## Join part 2

- Join the enriched Weather table with Flight dataset on both IATA = ORIGIN/DEST and the nearest weather Timestamp to two hours prior to flight departure time

In [0]:
# Join -- without the window function but with rounded timing
df_full_5 = df_airlines_pre_join \
  .join(df_weather_origin, on=[df_airlines_pre_join.utc_dep_3hrs_prior_rounded == df_weather_origin.DATE_WTHR_rounded_origin, df_airlines_pre_join.ORIGIN_AIRLNS == df_weather_origin.iata_origin], how='left') \
  .join(df_weather_dest, on=[df_airlines_pre_join.utc_dep_3hrs_prior_rounded == df_weather_dest.DATE_WTHR_rounded_dest, df_airlines_pre_join.DEST_AIRLNS == df_weather_dest.iata_dest], how='left')

# Now multiple weather station data could be joined to the same flight on either origin or dest - need to reduce that to only 1
windowSpec = Window.partitionBy("id_AIRLNS").orderBy(F.expr("bigint(DATE_WTHR_origin) - bigint(DATE_WTHR_rounded_origin) + bigint(DATE_WTHR_dest) - bigint(DATE_WTHR_rounded_dest)"))

df_full_5_dedup = df_full_5 \
  .withColumn("row_number", row_number().over(windowSpec)) \
  .filter("row_number == 1")


## Write to blob and check join

In [0]:
# df_full_5_dedup.write.parquet(f"{blob_url}/2015_q1_full_join_5")

# df_full_5_dedup.write.parquet(f"{blob_url}/all_time_full_join_1")

df_full_5_dedup.write.parquet(f"{blob_url}/esther/all_time_full_join") # New version: Included TAIL_NUM from Flight Dataset, filtered out Cancelled and diverted flights, created utc_arrive

In [0]:
df_airlines_pre_join.count()

Out[83]: 161057

In [0]:
df_full_5_dedup.cache()

Out[87]: DataFrame[YEAR_AIRLNS: int, QUARTER_AIRLNS: int, MONTH_AIRLNS: int, DAY_OF_MONTH_AIRLNS: int, DAY_OF_WEEK_AIRLNS: int, FL_DATE_AIRLNS: string, DEP_TIME_AIRLNS: int, DEP_TIME_BLK_AIRLNS: string, CRS_DEP_TIME_AIRLNS: int, CRS_ARR_TIME_AIRLNS: int, CRS_ELAPSED_TIME_AIRLNS: double, ARR_TIME_AIRLNS: int, ARR_TIME_BLK_AIRLNS: string, ACTUAL_ELAPSED_TIME_AIRLNS: double, ORIGIN_AIRLNS: string, ORIGIN_CITY_NAME_AIRLNS: string, ORIGIN_STATE_ABR_AIRLNS: string, ORIGIN_STATE_FIPS_AIRLNS: int, ORIGIN_STATE_NM_AIRLNS: string, ORIGIN_WAC_AIRLNS: int, DEST_AIRLNS: string, DEST_CITY_NAME_AIRLNS: string, DEST_STATE_ABR_AIRLNS: string, DEST_STATE_FIPS_AIRLNS: int, DEST_STATE_NM_AIRLNS: string, DEST_WAC_AIRLNS: int, OP_UNIQUE_CARRIER_AIRLNS: string, FLIGHTS_AIRLNS: double, DISTANCE_AIRLNS: double, DISTANCE_GROUP_AIRLNS: int, DIVERTED_AIRLNS: double, CANCELLED_AIRLNS: double, CANCELLATION_CODE_AIRLNS: string, CARRIER_DELAY_AIRLNS: double, DEP_DELAY_AIRLNS: double, DEP_DELAY_NEW_AIRLNS: double, DEP_DELAY_GROUP_AIRLNS: int, DEP_DEL15_AIRLNS: double, ARR_DELAY_AIRLNS: double, ARR_DELAY_NEW_AIRLNS: double, ARR_DELAY_GROUP_AIRLNS: int, ARR_DEL15_AIRLNS: double, WEATHER_DELAY_AIRLNS: double, NAS_DELAY_AIRLNS: double, SECURITY_DELAY_AIRLNS: double, LATE_AIRCRAFT_DELAY_AIRLNS: double, id_AIRLNS: bigint, datetime_dep: timestamp, iata_OPNFLGHT: string, tz_db_time_zone_OPNFLGHT: string, utc_dep: timestamp, utc_dep_3hrs_prior: timestamp, utc_dep_3hrs_prior_rounded: timestamp, STATION_WTHR_origin: string, SOURCE_WTHR_origin: smallint, DATE_WTHR_origin: timestamp, LATITUDE_WTHR_origin: double, LONGITUDE_WTHR_origin: double, ELEVATION_WTHR_origin: double, NAME_WTHR_origin: string, REPORT_TYPE_WTHR_origin: string, CALL_SIGN_WTHR_origin: string, QUALITY_CONTROL_WTHR_origin: string, WND_WTHR_origin: string, CIG_WTHR_origin: string, VIS_WTHR_origin: string, TMP_WTHR_origin: string, DEW_WTHR_origin: string, SLP_WTHR_origin: string, GA1_WTHR_origin: string, GF1_WTHR_origin: string, MA1_WTHR_origin: string, REM_WTHR_origin: string, AA1_WTHR_origin: string, AA2_WTHR_origin: string, AJ1_WTHR_origin: string, AL1_WTHR_origin: string, AN1_WTHR_origin: string, AO1_WTHR_origin: string, AU1_WTHR_origin: string, AT1_WTHR_origin: string, iata_origin: string, icao_origin: string, station_origin: string, tz_db_time_zone_origin: string, DATE_WTHR_rounded_origin: timestamp, STATION_WTHR_dest: string, SOURCE_WTHR_dest: smallint, DATE_WTHR_dest: timestamp, LATITUDE_WTHR_dest: double, LONGITUDE_WTHR_dest: double, ELEVATION_WTHR_dest: double, NAME_WTHR_dest: string, REPORT_TYPE_WTHR_dest: string, CALL_SIGN_WTHR_dest: string, QUALITY_CONTROL_WTHR_dest: string, WND_WTHR_dest: string, CIG_WTHR_dest: string, VIS_WTHR_dest: string, TMP_WTHR_dest: string, DEW_WTHR_dest: string, SLP_WTHR_dest: string, GA1_WTHR_dest: string, GF1_WTHR_dest: string, MA1_WTHR_dest: string, REM_WTHR_dest: string, AA1_WTHR_dest: string, AA2_WTHR_dest: string, AJ1_WTHR_dest: string, AL1_WTHR_dest: string, AN1_WTHR_dest: string, AO1_WTHR_dest: string, AU1_WTHR_dest: string, AT1_WTHR_dest: string, iata_dest: string, icao_dest: string, station_dest: string, tz_db_time_zone_dest: string, DATE_WTHR_rounded_dest: timestamp, row_number: int]

In [0]:
df_full_5_dedup.count()

Out[88]: 161057

In [0]:
df_full_joined = spark.read.parquet(f"{blob_url}/all_time_full_join_1")

In [0]:
df_full_joined.count()

Out[109]: 31746831

In [0]:
display(df_full_joined)

YEAR_AIRLNS,QUARTER_AIRLNS,MONTH_AIRLNS,DAY_OF_MONTH_AIRLNS,DAY_OF_WEEK_AIRLNS,FL_DATE_AIRLNS,DEP_TIME_AIRLNS,DEP_TIME_BLK_AIRLNS,CRS_DEP_TIME_AIRLNS,CRS_ARR_TIME_AIRLNS,CRS_ELAPSED_TIME_AIRLNS,ARR_TIME_AIRLNS,ARR_TIME_BLK_AIRLNS,ACTUAL_ELAPSED_TIME_AIRLNS,ORIGIN_AIRLNS,ORIGIN_CITY_NAME_AIRLNS,ORIGIN_STATE_ABR_AIRLNS,ORIGIN_STATE_FIPS_AIRLNS,ORIGIN_STATE_NM_AIRLNS,ORIGIN_WAC_AIRLNS,DEST_AIRLNS,DEST_CITY_NAME_AIRLNS,DEST_STATE_ABR_AIRLNS,DEST_STATE_FIPS_AIRLNS,DEST_STATE_NM_AIRLNS,DEST_WAC_AIRLNS,OP_UNIQUE_CARRIER_AIRLNS,FLIGHTS_AIRLNS,DISTANCE_AIRLNS,DISTANCE_GROUP_AIRLNS,DIVERTED_AIRLNS,CANCELLED_AIRLNS,CANCELLATION_CODE_AIRLNS,CARRIER_DELAY_AIRLNS,DEP_DELAY_AIRLNS,DEP_DELAY_NEW_AIRLNS,DEP_DELAY_GROUP_AIRLNS,DEP_DEL15_AIRLNS,ARR_DELAY_AIRLNS,ARR_DELAY_NEW_AIRLNS,ARR_DELAY_GROUP_AIRLNS,ARR_DEL15_AIRLNS,WEATHER_DELAY_AIRLNS,NAS_DELAY_AIRLNS,SECURITY_DELAY_AIRLNS,LATE_AIRCRAFT_DELAY_AIRLNS,id_AIRLNS,datetime_dep,iata_OPNFLGHT,tz_db_time_zone_OPNFLGHT,utc_dep,utc_dep_3hrs_prior,utc_dep_3hrs_prior_rounded,STATION_WTHR_origin,SOURCE_WTHR_origin,DATE_WTHR_origin,LATITUDE_WTHR_origin,LONGITUDE_WTHR_origin,ELEVATION_WTHR_origin,NAME_WTHR_origin,REPORT_TYPE_WTHR_origin,CALL_SIGN_WTHR_origin,QUALITY_CONTROL_WTHR_origin,WND_WTHR_origin,CIG_WTHR_origin,VIS_WTHR_origin,TMP_WTHR_origin,DEW_WTHR_origin,SLP_WTHR_origin,GA1_WTHR_origin,GF1_WTHR_origin,MA1_WTHR_origin,REM_WTHR_origin,AA1_WTHR_origin,AA2_WTHR_origin,AJ1_WTHR_origin,AL1_WTHR_origin,AN1_WTHR_origin,AO1_WTHR_origin,AU1_WTHR_origin,AT1_WTHR_origin,iata_origin,icao_origin,station_origin,tz_db_time_zone_origin,DATE_WTHR_rounded_origin,STATION_WTHR_dest,SOURCE_WTHR_dest,DATE_WTHR_dest,LATITUDE_WTHR_dest,LONGITUDE_WTHR_dest,ELEVATION_WTHR_dest,NAME_WTHR_dest,REPORT_TYPE_WTHR_dest,CALL_SIGN_WTHR_dest,QUALITY_CONTROL_WTHR_dest,WND_WTHR_dest,CIG_WTHR_dest,VIS_WTHR_dest,TMP_WTHR_dest,DEW_WTHR_dest,SLP_WTHR_dest,GA1_WTHR_dest,GF1_WTHR_dest,MA1_WTHR_dest,REM_WTHR_dest,AA1_WTHR_dest,AA2_WTHR_dest,AJ1_WTHR_dest,AL1_WTHR_dest,AN1_WTHR_dest,AO1_WTHR_dest,AU1_WTHR_dest,AT1_WTHR_dest,iata_dest,icao_dest,station_dest,tz_db_time_zone_dest,DATE_WTHR_rounded_dest,row_number
2019,4,10,5,6,2019-10-05,1435,1400-1459,1440,1605,145.0,1550,1600-1659,135.0,MHT,"Manchester, NH",NH,33,New Hampshire,14,MDW,"Chicago, IL",IL,17,Illinois,41,WN,1.0,838.0,4,0.0,0.0,null,null,-5.0,0.0,-1,0.0,-15.0,0.0,-1,0.0,null,null,null,null,948,2019-10-05T14:40:00.000+0000,MHT,America/New_York,2019-10-05T18:40:00.000+0000,2019-10-05T15:40:00.000+0000,2019-10-05T15:00:00.000+0000,74394514710,7,2019-10-05T15:53:00.000+0000,42.92963,-71.43566,67.4,"MANCHESTER AIRPORT, NH US",FM-15,KMHT,V020,"180,5,N,0026,5","22000,5,9,N","016093,5,N,5","+0122,5","-0017,5","10318,5","00,5,+99999,9,99,9","00,99,1,99,9,99,9,99999,9,99,9,99,9","10305,5,10219,5",MET09610/05/19 10:53:02 METAR KMHT 051553Z 18005KT 10SM CLR 12/M02 A3043 RMK AO2 SLP318 T01221017 (CZ),"01,0000,9,5",,,,,,,,MHT,KMHT,74394514710,America/New_York,2019-10-05T15:00:00.000+0000,72534014819,4,2019-10-05T15:00:00.000+0000,41.78611,-87.75222,186.5,"CHICAGO MIDWAY AIRPORT, IL US",FM-12,99999,V020,"150,1,N,0062,1","99999,9,9,N","016000,1,9,9","+0139,1","+0072,1","10203,1","99,9,+01250,1,99,9","06,99,1,99,9,99,9,01250,1,99,9,99,9","99999,9,09978,1",SYN06472534 32666 61512 10139 20072 39978 40203 58024 91453 555 90515=,,,,,,,,,MDW,KMDW,72534014819,America/Chicago,2019-10-05T15:00:00.000+0000,1
2019,4,10,26,6,2019-10-26,925,0900-0959,932,1048,196.0,1033,1000-1059,188.0,DFW,"Dallas/Fort Worth, TX",TX,48,Texas,74,SNA,"Santa Ana, CA",CA,6,California,91,AA,1.0,1205.0,5,0.0,0.0,null,null,-7.0,0.0,-1,0.0,-15.0,0.0,-1,0.0,null,null,null,null,1021,2019-10-26T09:32:00.000+0000,DFW,America/Chicago,2019-10-26T14:32:00.000+0000,2019-10-26T11:32:00.000+0000,2019-10-26T11:00:00.000+0000,72259003927,4,2019-10-26T11:53:00.000+0000,32.8978,-97.0189,170.7,"DAL FTW WSCMO AIRPORT, TX US",FM-15,99999,V020,"300,1,N,0077,1","00274,1,9,N","016093,1,9,9","+0089,1","+0072,1","10122,1",,"99,99,9,99,9,99,9,00274,1,99,9